# SI 618: Data Manipulation and Analysis
## 08 - Clustering
### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a> This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

### <font color="magenta">Q1: Record your music preferences</font>

Fill in your music preferences on https://docs.google.com/spreadsheets/d/1KY8tBiiSeehMDxXc4GU_mOoD8MRnUVkUtinadNJEgrU/edit?usp=sharing

Values should range from 1 to 10.  1=no way; 10=the best.  Please fill in a value for each column.

We will be downloading the completed sheet to use as data for this class.


![](resources/clustering/Slide01.png)

![](resources/clustering/Slide02.png)

![](resources/clustering/Slide03.png)

![](resources/clustering/Slide01.png)

![](resources/clustering/Slide04.png)

![](resources/clustering/Slide07.png)

![](resources/clustering/Slide09.png)

![](resources/clustering/Slide10.png)

![](resources/clustering/Slide11.png)

![](resources/clustering/Slide12.png)

![](resources/clustering/Slide13.png)

![](resources/clustering/Slide14.png)

![](resources/clustering/Slide15.png)

![](resources/clustering/Slide16.png)

![](resources/clustering/Slide17.png)

![](resources/clustering/Slide18.png)

![](resources/clustering/Slide19.png)

![](resources/clustering/Slide20.png)

![](resources/clustering/Slide21.png)

![](resources/clustering/Slide22.png)

![](resources/clustering/Slide23.png)

![](resources/clustering/Slide24.png)

![](resources/clustering/Slide25.png)

## NOW: Download music_wn2019.csv from Slack

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import load_iris
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc

import pandas as pd
import seaborn as sns

In [ ]:
music = pd.read_csv('data/music_wn2019.csv')

In [ ]:
music.head()

In [ ]:
music = music.dropna() # Clustering doesn't like NaNs
music_indexed_by_name = music.set_index('Name')

In [ ]:
music_indexed_by_name.head()

In [ ]:
model = AgglomerativeClustering(n_clusters=3) # we can make up the value for clusters
model.fit_predict(music_indexed_by_name)

### Plot Hierarachical Clustering Dendrogram 
The following function plots the corresponding dendrogram of a hierarchical clustering
using AgglomerativeClustering and the dendrogram method available in scipy.

It was writte by Mathew Kallada and modified by Chris Teplovs.
It is licensed under the BSD clause.

In [ ]:
import numpy as np

from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

def plot_dendrogram(model, **kwargs):

    # Children of hierarchical clustering
    children = model.children_

    # Distances between each pair of children
    # Since we don't have this information, we can use a uniform one for plotting
    distance = np.arange(children.shape[0])

    # The number of observations contained in each cluster level
    no_of_observations = np.arange(2, children.shape[0]+2)

    # Create linkage matrix and then plot the dendrogram
    linkage_matrix = np.column_stack([children, distance, no_of_observations]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)



In [ ]:
model = AgglomerativeClustering(n_clusters=3)
model = model.fit(music_indexed_by_name)

In [ ]:
plt.title('Hierarchical Clustering Dendrogram (People)')
plot_dendrogram(model,labels=music_indexed_by_name.index.values)
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.title('Hierarchical Clustering Dendrogram (Cluster Number)')
plot_dendrogram(model,labels=model.labels_)
plt.xticks(rotation=0)
plt.show()

In [ ]:
plt.title('Hierarchical Clustering Dendrogram (Row Number)')
plot_dendrogram(model)
plt.xticks(rotation=0)
plt.show()

### <font color="magenta">Q2: What's the "best" choice for the number of clusters?  What are your options? How would you choose? </font>

Insert your answer here

# K-Means Clustering

![](resources/clustering/Slide26.png)

![](resources/clustering/Slide27.png)

![](resources/clustering/Slide28.png)

![](resources/clustering/Slide31.png)

![](resources/clustering/Slide32.png)

![](resources/clustering/Slide33.png)

![](resources/clustering/Slide34.png)

![](resources/clustering/Slide36.png)

![](resources/clustering/Slide37.png)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
kmeans = KMeans(n_clusters=3) # start with 3 clusters
kmeans.fit(music_indexed_by_name)
print("Labels:")
print(kmeans.labels_)


In [ ]:
data = music_indexed_by_name.copy()

And now let's join the labels with the original dataframe:

In [ ]:
data_joined = pd.concat([pd.DataFrame(kmeans.labels_).reset_index(),music.reset_index()],axis=1).drop('index',axis=1)

In [ ]:
data_joined.head()

## Picking K – How many clusters?

A number of clustering methods, such as k-means, assumes the parameter k (#clusters) is known in advance, which is often not the case in practice. A number of techniques exist for determining the number of clusters in a dataset. See [this Wikipedia page](https://en.wikipedia.org/wiki/Determining_the_number_of_clusters_in_a_data_set) for a detailed discussion.

In this section, we focus on four of the approaches:

1. Educated guessing
	* What are some criteria you could use?
2. Rule of thumb
3. The Elbow Method
4. The Silhouette Approach

Let us see if all the methods listed above will be able to recover the true number of clusters.

### Rule of Thumb

By rule of thumb, we could potentially choose the number of clusters as simply


$$ k \approx \sqrt{n/2} $$

where $n$ is the number of observations.

### The Elbow Method (Scree Plot)

Here, we plot the percentage of variance explained as a function of the number of clusters.

As we increase *k*, this percentage will naturally increase. Where we see diminishing returns to increasing *k* is a good place to stop, which corresponds to the "elbow" in the graph. This is a bit of an art, since the elbow is not unambiguous.


### The Silhouette Method

The silhouette coefficient is a measure of how closely a particular datum is matched to data and how loosely it is matched to data of that neighboring cluster.

You'd therefore expect that a datum that's a good match for its cluster to have a silhouette of 1, and a data that's a poor match to have a score of -1.

We can find the average silhouette of the data for particular values of *k*, and choose the maximum.


In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

In [ ]:
pca = PCA(n_components=2).fit(data)
data2d = pca.transform(data)

plt.figure(figsize=(16, 8))
scores, n_clusters, preds = [], [], []
for i in range(2, 10):
    kmean = KMeans(n_clusters = i).fit(data)
    scores.append(kmean.score(data))
    n_clusters.append(i)
    pred = kmean.predict(data)
    preds.append(pred)
    plt.subplot(2, 4, i - 1)
    plt.title(f"{i} clusters silhoute={np.round(silhouette_score(data, pred), decimals=5)}")
    plt.scatter(data2d[:, 0], data2d[:, 1], c=pred, marker = '.')
    
    centroids = kmean.cluster_centers_
    centroids2d = pca.transform(centroids)
    plt.plot(centroids2d[:, 0], centroids2d[:, 1], 'b+', markersize=15)

### Example: Use these three methods (rule of thumb, elbow, and silhouette) to determine the ideal number of clusters for a k-means clustering of our music preferences data.</font>


Hints:

* Use inertia, the within-cluster sum of squares as the criterion for the elbow method. This is [available](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) in your `k_means` result as `k_means.inertia_`.
* The silhouette score can be computed with [sklearn.metrics.silhouette\_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score). `silhouette_score` takes two arguments: `data` and `k_means.labels_`.

In [ ]:
np.sqrt(len(data)/2)

In [ ]:
from sklearn import metrics

cost = []
for k in range(2,16):
    k_means = KMeans(init='k-means++', n_clusters=k, n_init=10)
    k_means.fit(data)
    cost.append(k_means.inertia_)

fig, ax = plt.subplots()
plt.plot(range(2,16), cost, 'b*-')
plt.xlim(1, plt.xlim()[1])


In [ ]:
cost = []
for k in range(2,16):
    k_means = KMeans(init='k-means++', n_clusters=k, n_init=10)
    k_means.fit(data)
    cost.append(metrics.silhouette_score(data, k_means.labels_))

# kIdx = np.argmax(cost)
fig, ax = plt.subplots()
plt.plot(range(2,16), cost, 'b*-')
plt.xlim(1, plt.xlim()[1])

What did you get? Did they all agree?

# BREAK

## Applying what we just learned to a Pokemon dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### <font color="magenta">Q3: Study and describe the dataset at https://www.kaggle.com/alopez247/pokemon</a>
The data file has been provided in today's zip file as data/pokemon_alopez247.csv.

Provide a detailed description of the dataset in your own words.  Include items such as the shape of the dataset (i.e. the number of rows and columns), the number of continuous variables, the number of categorical variables, the number of boolean variables, and so on.  Provide graphical representations of the data (e.g. histograms, bar charts, scatterplots, etc.) where appropriate.

In [ ]:
pokemon = pd.read_csv('data/pokemon_alopez247.csv',index_col='Name')

In [ ]:
pokemon.head()

#### INSERT YOUR DETAILED DESCRIPTION HERE

### <font color="magenta">Q4: Create dummy variables for categorical variables.  </font>
We will discuss why you need to use ```drop_first=True``` argument to ```pd.get_dummies``` in class.  What is the shape of the resulting DataFrame?


### <font color="magenta">Q5: Fit a 3-cluster agglomerative clustering model to the pokemon data (with dummy variables).</a>
Provide a graphical representation of the results and save the figure as a PDF file.  Provide an interpretation of your results

In [ ]:
# Insert your code here

Insert your interpretations here.

### <font color="magenta">Q6: Fit a series of k-means models to the Pokemon dataset.</font>

**Your values of k should range from 2 to 9.**

Select the optimal number of *k*.

Use the four methods suggested above (i.e. educated guess, rule of thumb, elbow method, and silhouette.  Provide plots for the elbow and silhouette methods.  Include 2-D plots of the centroids and individual data points for a reasonable range of *k* values (we suggest 2 to 9).  What, in your opinion, is the best choice for *k*?  What are the characteristics of each of the clusters in your solution (i.e. what characteristics do the members of each cluster have in common?)


In [ ]:
# insert your code here

Insert your interpretations here.

## END OF NOTEBOOK
Please remember to submit your notebook in both .HTML and .IPYNB formats via Canvas.